<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/new_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [4]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.file-revisions-by-id
MyDrive
.shortcut-targets-by-id
.Trash


In [5]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-16 08:26:20--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162863 (1.1M) [text/plain]
Saving to: ‘coref_df_anns60k.csv’

coref_df_anns60k.cs 100%[===================>]   1.11M  --.-KB/s    in 0.02s   

2021-05-16 08:26:20 (49.6 MB/s) - ‘coref_df_anns60k.csv’ saved [1162863/1162863]

--2021-05-16 08:26:20--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request s

In [47]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split

In [30]:
with open('coref_df_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else '0' for i in row]
      # print(new_row)
      if len(new_row) > 9:
        # print(new_row[:9])
        new_row = new_row[:9]
      writer.writerow(new_row)


In [39]:
files.download('new_df_anns60k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

In [33]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1,0,0,0,0,обсуждаем,0,0.0
1,препарат,1,0,0,1,0,обсуждаем,0,0.0
2,рисперидон,1,0,0,2,0,препарат,1,2.0
3,рисполепт,1,0,0,3,0,рисперидон,1,3.0
4,сперидан,1,0,0,4,0,рисполепт,1,4.0
...,...,...,...,...,...,...,...,...,...
26492,и,22,68,0,2385,0,пошлины,0,0.0
26493,таможенные,22,68,0,2386,0,пошлины,0,0.0
26494,пошлины,22,68,0,2387,0,есть,0,0.0
26495,как,22,68,0,2388,0,есть,0,0.0


In [34]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')

In [35]:
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [40]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0.0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0.0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,1,2.0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,1,3.0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0.0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0.0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0.0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0.0


In [43]:
cor_df.iloc[[6541]]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
6541,доминанту,[ 0.04290138 0.2685304 0.35882333 -0.046137...,1,5,17,0,339,inan,accs,femn,NaN,NOUN,obj,убирает,NaN,NaN,NaN,VERB,[-0.0018607 0.9945493 0.8423873 -0.306027...,[ 4.53790314e-02 1.01628333e-01 -3.12765948e-...,[ 0.18719818 0.38022175 0.22736616 0.077237...,0,0.0


In [44]:
cor_df.iloc[:2,:]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0.0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0.0


In [45]:
df = cor_df.iloc[:6541,:]

In [ ]:
cor_df.iloc[:1,:]

In [65]:
len(df.loc[df['NER'] == 1])

283

In [66]:
len(df.loc[df['NER'] == 2])

89

In [67]:
len(df.loc[df['NER'] == 3])

54

In [70]:
len(df.loc[df['NER'].isin([1,2,3])])

426

In [ ]:
df.loc[df['NER'].isin([1,2,3])]

In [63]:
df.describe()

,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,NER,COREFERENCE_CLUSTER
count,6541.000000,6541.000000,6541.00000,6541.000000,6541.000000,6541.000000,6541.000000
mean,0.929369,2.830148,15.74377,4.258829,1180.104877,0.095245,66.903532
std,0.256228,0.910212,12.06082,5.718161,1106.784078,0.403665,370.716460
min,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,6.00000,1.000000,234.000000,0.000000,0.000000
50%,1.000000,3.000000,14.00000,2.000000,800.000000,0.000000,0.000000
75%,1.000000,3.000000,23.00000,5.000000,2033.000000,0.000000,0.000000
max,1.000000,5.000000,49.00000,31.000000,3668.000000,3.000000,3653.000000


In [48]:
train, test = train_test_split(df, test_size=0.2)

In [50]:
train

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
5094,не,[ 1.3166505 0.7812625 0.03431194 0.549083...,1,3,37,2,3210,NaN,NaN,NaN,NaN,PRCL,advmod,равно,NaN,NaN,NaN,CONJ,[ 0.6384062 -0.2530158 0.4628007 0.504675...,[ 6.81811618e-03 2.16187220e-02 1.59193221e-...,[ 0.5665885 0.11021133 -0.00343395 0.366016...,0,0.0
4693,возникновения,[ 0.8664403 0.40672255 0.00478971 -0.141247...,1,3,32,5,2809,inan,gent,neut,NaN,NOUN,nmod,профилактика,inan,nomn,femn,NOUN,[ 0.47712106 1.4062586 0.41634303 0.477816...,[ 0.0203335 0.04634707 0.0050947 -0.006470...,[ 0.8566508 0.6194427 -0.06817464 0.148334...,0,0.0
5647,не,[ 1.3166505 0.7812625 0.03431194 0.549083...,1,4,4,0,23,NaN,NaN,NaN,NaN,PRCL,advmod,принимаю,NaN,NaN,NaN,VERB,[ 0.05930857 0.74574506 0.42736983 -0.408226...,[ 1.02194131e-03 1.94714397e-01 3.48628536e-...,[ 0.38121995 0.42492685 0.120866 0.149430...,0,0.0
3359,не,[ 1.3166505 0.7812625 0.03431194 0.549083...,1,3,18,7,1475,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,[ 0.59691274 0.32549492 0.28281662 -0.178887...,[ 0.00463667 0.04266815 0.02191578 0.006892...,[ 0.4862453 0.559837 -0.16834979 -0.177250...,0,0.0
534,лечили,[ 0.98267573 0.39160138 -0.7314423 0.513561...,1,1,9,1,254,NaN,NaN,NaN,NaN,VERB,ROOT,лечили,NaN,NaN,NaN,VERB,[ 0.98267573 0.39160138 -0.7314423 0.513561...,[ 0.08315846 0.16683042 0.07626291 -0.058545...,[ 0.18342188 0.5628554 0.3697557 -0.437373...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,несколько,[ 0.7079202 -0.01007268 0.02112754 0.060836...,1,2,17,2,946,NaN,NaN,NaN,NaN,ADVB,nummod:gov,сонлив,NaN,NaN,masc,ADJS,[ 0.51633596 0.67358726 0.47839665 -0.208238...,[ 0.05917258 0.06837457 0.04875167 -0.020298...,[ 0.4823186 0.36119863 0.25625664 0.200282...,0,0.0
2693,вопрос,[ 0.41350657 1.0234988 0.6429038 -0.423172...,1,3,12,0,809,inan,nomn,masc,NaN,NOUN,ROOT,вопрос,inan,nomn,masc,NOUN,[ 0.41350657 1.0234988 0.6429038 -0.423172...,[ 8.83663818e-03 2.38674413e-02 1.33625148e-...,[ 0.36075425 0.59063596 -0.24404798 0.278417...,0,0.0
3829,28,[-0.26670137 -0.205963 0.3164352 -0.635385...,1,3,23,4,1945,NaN,NaN,NaN,NaN,NaN,nummod,szt,NaN,NaN,NaN,NaN,[-0.11868086 -0.05016071 0.1532267 -0.274004...,[-4.23500023e-04 -1.25278442e-04 6.87614141e-...,[-0.07967585 -0.07066966 -0.03800365 -0.053264...,0,0.0
2284,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,1,3,5,12,400,NaN,NaN,NaN,NaN,CONJ,cc,идет,NaN,NaN,NaN,VERB,[ 8.35733175e-01 5.94675660e-01 -2.10102320e-...,[ 3.43929487e-03 8.13973416e-03 -6.64101914e-...,[ 0.6446236 0.42039558 0.01956174 -0.048540...,0,0.0


In [ ]:
# X = cor_df.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# X_test = cor_df_test.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [ ]:
X

In [ ]:
labels_dict['HEAD_POS']

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X, y)

In [ ]:
clf_ent = tree.DecisionTreeClassifier(criterion='entropy')
clf_ent.fit(X, y)

In [ ]:
from graphviz import Source
from sklearn import tree

In [ ]:
# tree.plot_tree(clf)

In [ ]:
# graph = Source(tree.export_graphviz(clf, out_file=None, 
#                                     feature_names=list(X), class_names=uniques, filled=True))

# display(SVG(graph.pipe(format='svg')))

In [ ]:
clf.predict(X_test)

In [ ]:
y_test

In [ ]:
X_test